TSG035 - Spark History logs
===========================

Description
-----------

Steps
-----

### Parameters

In [ ]:
import re

tail_lines = 2000

pod = None # All
container='hadoop-livy-sparkhistory'
log_files = [ "/var/log/supervisor/log/sparkhistory*" ]

expressions_to_analyze = [
    re.compile(".{23} WARN "),
    re.compile(".{23} ERROR ")
]

### Instantiate Kubernetes client

In [ ]:
# Instantiate the Python Kubernetes client into 'api' variable

import os

try:
    from kubernetes import client, config
    from kubernetes.stream import stream

    if "KUBERNETES_SERVICE_PORT" in os.environ and "KUBERNETES_SERVICE_HOST" in os.environ:
        config.load_incluster_config()
    else:
        config.load_kube_config()

    api = client.CoreV1Api()

    print('Kubernetes client instantiated')
except ImportError:
    from IPython.display import Markdown
    display(Markdown(f'SUGGEST: Use [SOP059 - Install Kubernetes Python module](../install/sop059-install-kubernetes-module.ipynb) to resolve this issue.'))
    raise

### Get the namespace for the big data cluster

Get the namespace of the big data cluster from the Kuberenetes API.

NOTE: If there is more than one big data cluster in the target
Kubernetes cluster, then set \[0\] to the correct value for the big data
cluster.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

try:
    namespace = api.list_namespace(label_selector='MSSQL_CLUSTER').items[0].metadata.name
except IndexError:
    from IPython.display import Markdown
    display(Markdown(f'SUGGEST: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
    display(Markdown(f'SUGGEST: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
    display(Markdown(f'SUGGEST: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
    raise

print('The kubernetes namespace for your big data cluster is: ' + namespace)

### Get tail for log

In [ ]:
# Display the last 'tail_lines' of files in 'log_files' list

pods = api.list_namespaced_pod(namespace)

entries_for_analysis = []

for p in pods.items:
    if pod is None or p.metadata.name == pod:
        for c in p.spec.containers:
            if container is None or c.name == container:
                for log_file in log_files:
                    print (f"- LOGS: '{log_file}' for CONTAINER: '{c.name}' in POD: '{p.metadata.name}'")
                    try:
                        output = stream(api.connect_get_namespaced_pod_exec, p.metadata.name, namespace, command=['/bin/sh', '-c', f'tail -n {tail_lines} {log_file}'], container=c.name, stderr=True, stdout=True)
                    except Exception:
                        print (f"FAILED to get LOGS for CONTAINER: {c.name} in POD: {p.metadata.name}")
                    else:
                        for line in output.split('\n'):
                            for expression in expressions_to_analyze:
                                if expression.match(line):
                                    entries_for_analysis.append(line)
                            print(line)
print("")
print(f"{len(entries_for_analysis)} log entries found for further analysis.")

### Analyze log entries and suggest relevant Troubleshooting Guides

In [ ]:
# Analyze log entries and suggest further relevant troubleshooting guides

from IPython.display import Markdown

tsgs = []

suggestions = 0
for entry in entries_for_analysis:
    print (entry)

    for tsg in tsgs:
        if entry.find(tsg[0]) != -1:
            display(Markdown(f'SUGGEST: Use [{tsg[2]}](tsg[1]) to resolve this issue.'))
            suggestions = suggestions + 1

print("")
print(f"{len(entries_for_analysis)} log entries analyzed. {suggestions} further troubleshooting suggestions made inline.")

In [ ]:
print('Notebook execution complete.')